<a href="https://colab.research.google.com/github/sayprincekumar20/NLP-Quara-Pairs-Duplicates/blob/main/only_bow_implement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
df= pd.read_csv('questions.csv')

In [15]:
df.shape

(404351, 6)

In [16]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [17]:
# trim data so that model run on my system smoothly:30000
new_df= df.sample(30000)

In [18]:
new_df.isnull().sum()

,0
id,0
qid1,0
qid2,0
question1,0
question2,0
is_duplicate,0


In [19]:
new_df.duplicated().sum()

np.int64(0)

In [20]:
ques_df= new_df[['question1','question2']]
ques_df.head()

,question1,question2
221308,What is it like living in Sydney for an Indian?,How is life for an Indian in Sydney?
156533,Is ethics a subject of discussion in some area...,What are mathematics intensive research areas ...
352506,Why companies like QNET are not being banned i...,Was the demonetization of money in India a boo...
111073,"Where, if anywhere, in the Qur'an does it say ...",What does the Koran say about non-believers?
293227,How do you start a conversation?,How do I start a conversation?


In [22]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions= list(ques_df['question1'])+ list(ques_df['question2'])
cv= CountVectorizer(max_features=3000)
q1_arr,q2_arr= np.vsplit(cv.fit_transform(questions).toarray(),2)

In [23]:
temp_df1= pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2= pd.DataFrame(q2_arr, index= ques_df.index)
temp_df= pd.concat([temp_df1,temp_df2], axis=1)
temp_df.shape

(30000, 6000)

In [24]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
221308,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
156533,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
352506,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
111073,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
293227,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322316,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
133031,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17148,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
78139,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
temp_df['is_duplicate']= new_df['is_duplicate']

In [26]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
221308,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
156533,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
352506,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
111073,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
293227,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [28]:
from sklearn.model_selection import train_test_split
X_train, X_text, y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values, temp_df.iloc[:,-1].values, test_size=0.2, random_state=1)


In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf= RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred= rf.predict(X_text)
accuracy_score(y_test, y_pred)

0.7476666666666667

In [31]:
from xgboost import XGBClassifier
xgb= XGBClassifier()
xgb.fit(X_train, y_train)
y_pred= xgb.predict(X_text)
accuracy_score(y_test, y_pred)

0.729